In [1]:
include("../LiPoSID.jl")
using PyPlot

Main.LiPoSID

In [11]:
data_dir = "../DATA/"
γ_list = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]

function bloch4(ρ)

    b = convert.(Float64, [ ρ[1,2] + ρ[2,1],
                           (ρ[1,2] - ρ[2,1])*im,    #ρ[2,1] - ρ[1,2] ?
                            ρ[1,1] - ρ[2,2],
                               1                 ]) #ρ[1,1] + ρ[2,2]  

end

function dm_b4(b) 

    ρ = [ 1+b[3]         b[1]-im*b[2]
          b[1]+im*b[2]   1-b[3]       ]/2

end

function propagate_DMD_b4(A, ρ₀, lᵐᵃˣ)

    ρ = [ρ₀]

    for i in 2:lᵐᵃˣ
        push!(ρ, dm_b4( A * bloch4(ρ[end])))
    end

    return ρ

end

function DMD_step(A, ρ₀)
    dm_b4(A * bloch4(ρ₀))
end

max_neg(d) = minimum(d) < 0 ? abs(minimum(d)) : 0


max_neg (generic function with 1 method)

In [12]:
ρᵍ₀ = [1. 0]' * [1 0]
ρᵉ₀ = [0. 1]' * [0 1]
ρˣ₀ = [1  1]' * [1 1]/2
ρʸ₀ = [1 -im]' * [1 -im]/2

negs=[]

for γᵢ in γ_list 

    ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵢ)
    ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵢ)
    ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵢ)
    ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵢ)

    lᵉ = length(ρᵉ); lᵍ = length(ρᵍ); lˣ = length(ρˣ); lʸ = length(ρʸ)
    lᵐᵃˣ = min(lᵉ, lᵍ,  lˣ, lʸ)  #choose time limit by shortest series

    Aᴰᴹᴰ⁻ˢᴮ = LiPoSID.direct_DMD_01XY_b4_A([ρᵉ[1:lᵐᵃˣ], ρᵍ[1:lᵐᵃˣ], ρˣ[1:lᵐᵃˣ], ρʸ[1:lᵐᵃˣ]])

    ρᵍ₁ˢᴮ = DMD_step(Aᴰᴹᴰ⁻ˢᴮ, ρᵍ₀)
    ρᵉ₁ˢᴮ = DMD_step(Aᴰᴹᴰ⁻ˢᴮ, ρᵉ₀)
    ρˣ₁ˢᴮ = DMD_step(Aᴰᴹᴰ⁻ˢᴮ, ρˣ₀)
    ρʸ₁ˢᴮ = DMD_step(Aᴰᴹᴰ⁻ˢᴮ, ρʸ₀)

    Eᴰᴹᴰ⁻ˢᴮ, dˢᴮ = LiPoSID.QPT(ρᵍ₁ˢᴮ, ρᵉ₁ˢᴮ, ρˣ₁ˢᴮ, ρʸ₁ˢᴮ)

    push!(negs, max_neg(dˢᴮ))

end

In [17]:
plot(negs)
PyPlot.show();

In [18]:
arr = [2, 5, 6, 8]; 
arr_2 = arr * 3; 

plot(arr, arr_2, linewidth = 5.0, 
     linestyle = "--", color = "green"); 
  
title("A simple Line Chart"); 
xlabel("This is X-Axis"); 
ylabel("This is Y-Axis"); 
  
PyPlot.show();